# Stellar Origin Profiles

In [60]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import binned_statistic

In [61]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup
from auriga.settings import Settings

In [62]:
figure_setup()

In [63]:
settings = Settings()

In [64]:
def read_data(simulation: str) -> pd.DataFrame:
    settings = Settings()
    
    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.tag_particles_by_region(
        disc_std_circ=settings.disc_std_circ,
        disc_min_circ=settings.disc_min_circ,
        cold_disc_delta_circ=settings.cold_disc_delta_circ,
        bulge_max_specific_energy=settings.bulge_max_specific_energy)
    s.add_extra_coordinates()
    s.tag_in_situ_stars()

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    df = pd.DataFrame({
        "ComponentTag": s.region_tag[is_real_star & is_main_obj],
        "CylindricalRadius_ckpc": s.rho[is_real_star & is_main_obj],
        "SphericalRadius_ckpc": s.r[is_real_star & is_main_obj],
        "IsInSitu": s.is_in_situ[is_real_star & is_main_obj],
        })

    return df

In [65]:
def create_stats_df(df: pd.DataFrame,
                    rxy_range: tuple = (0, 50),
                    n_bins: int = 50) -> pd.DataFrame:
    n_stars, bin_edges = np.histogram(
        a=df["CylindricalRadius_ckpc"],
        range=rxy_range, bins=n_bins)
    n_insitu, _ = np.histogram(
        a=df["CylindricalRadius_ckpc"][df["IsInSitu"] == 1],
        range=rxy_range, bins=n_bins)
    bin_centers = bin_edges[1:] - np.diff(bin_edges) / 2

    stats = {}
    stats["BinCenters_ckpc"] = bin_centers
    stats["InSituFraction"] = n_insitu / n_stars

    for i in range(len(settings.components)):
        n_stars, _ = np.histogram(
            a=df["CylindricalRadius_ckpc"][
                df["ComponentTag"] == i],
            range=rxy_range, bins=n_bins)
        n_insitu, _ = np.histogram(
            a=df["CylindricalRadius_ckpc"][
                (df["IsInSitu"] == 1) & (df["ComponentTag"] == i)],
            range=rxy_range, bins=n_bins)
        stats[f"InSituFraction_{settings.components[i]}"] = n_insitu / n_stars

    return pd.DataFrame(stats)

In [66]:
for i in settings.galaxies:
    SIMULATION = f"au{i}_or_l4_s127"
    df = read_data(simulation=SIMULATION)
    stats = create_stats_df(df=df)

    fig = plt.figure(figsize=(3.0, 2.5))
    gs = fig.add_gridspec(nrows=1, ncols=1, hspace=0.0, wspace=0.0)
    ax = gs.subplots(sharex=True, sharey=True)

    ax.set_xlabel(r"$r_{xy}$ [ckpc]")
    ax.set_xlim(0, 50)
    ax.set_xticks([0, 10, 20, 30, 40, 50])
    ax.set_ylabel(r"$f_\mathrm{in-situ}$")
    ax.set_ylim(0, 1)
    ax.set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1])

    ax.plot(stats["BinCenters_ckpc"], stats["InSituFraction"],
            c="black", label="All")
    for i in range(len(settings.components)):
        ax.plot(
            stats["BinCenters_ckpc"],
            stats[f"InSituFraction_{settings.components[i]}"],
            c=settings.component_colors[settings.components[i]],
            label=settings.component_labels[settings.components[i]])

    ax.legend(loc="lower left", framealpha=0, fontsize=6.0)

    ax.text(x=ax.get_xlim()[0], y=ax.get_ylim()[1],
            s=r"$\texttt{" + SIMULATION.upper() + "}$",
            size=8.0, ha='left', va='bottom',
            )

    fig.savefig(f"../images/stellar_origin_profiles/{SIMULATION}.pdf")
    plt.close(fig)

/u/fiza/conda-envs/auriga/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide
/u/fiza/conda-envs/auriga/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide
/u/fiza/conda-envs/auriga/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide
/u/fiza/conda-envs/auriga/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide
/u/fiza/conda-envs/auriga/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide
/u/fiza/conda-envs/auriga/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide
/u/fiza/conda-envs/auriga/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide
/u/fiza/conda-envs/auriga/lib/python3.7/s